In [2]:
import torch 
import torch.nn.functional as F
from torch import nn

In [3]:
class Wisdom(nn.Module):
    def __init__(self):
        super().__init__()
       
    def forward(self, input):
        output = input + 1
        return output
    
wisdom = Wisdom()
x = torch.tensor(1.0)
output = wisdom(x)
print(output)

tensor(2.)


学习module的使用，定义一个类

In [5]:
#input 为输入图像
input = torch.tensor([[1,2,0,3,1],
                      [0,1,2,3,1],
                      [1,2,1,0,0],
                      [5,2,3,1,1],
                      [2,1,0,1,1]])

# kernel 为卷积核，
kernel = torch.tensor([[1,2,1],
                       [0,1,0],
                       [2,1,0]])

input = torch.reshape(input,(1,1,5,5))  # 1 为batch_size, 1 为通道数
# batch_size指一次训练迭代中，模型处理的样本数量，通道数表示特征图的数量

kernel = torch.reshape(kernel,(1,1,3,3))  

output = F.conv2d(input, kernel, stride=1) # stride为步长,表示卷积核每次移动的距离
print(output)

output2 = F.conv2d(input, kernel, stride=2) # 卷积核每次移动的距离为2
print(output2)

output3 = F.conv2d(input, kernel, stride=1, padding=1) # padding为填充, 1表示在图像周围填充一圈0,再进行卷积操作
print(output3)

tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])
tensor([[[[10, 12],
          [13,  3]]]])
tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])


学习卷积操作

KeyboardInterrupt: 